In [21]:
import numpy as np
import pandas as pd

In [22]:
fake_news = pd.read_csv('FakeNews/Fake.csv')
true_news = pd.read_csv('FakeNews/True.csv')

In [23]:
fake_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


In [24]:
true_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


In [25]:
fake_news.describe()

,title,text,subject,date
count,23481,23481,23481,23481
unique,17903,17455,6,1681
top,MEDIA IGNORES Time That Bill Clinton FIRED His...,,News,"May 10, 2017"
freq,6,626,9050,46


In [26]:
fake_news.subject.unique()

array(['News', 'politics', 'Government News', 'left-news', 'US_News',
       'Middle-east'], dtype=object)

In [27]:
true_news.subject.unique()

array(['politicsNews', 'worldnews'], dtype=object)

In [28]:
from data import PreProcessor

preproc = PreProcessor()

fake_news['title'].apply(preproc.forward)
fake_news['text'].apply(preproc.forward)
true_news['title'].apply(preproc.forward)
true_news['text'].apply(preproc.forward)

fake_news['title'].head()
#ERRORS HERE

0    donald trump send embarrassing new year eve me...
1    drunk bragging trump staffer start russian col...
2    sheriff david clarke become internet joke thre...
3      trump obsess even obama name code website image
4      pope francis call donald trump christmas speech
Name: title, dtype: object

In [29]:
fake_news = fake_news.assign(is_fake = [1]*len(fake_news))
true_news = true_news.assign(is_fake = [0]*len(true_news))

In [32]:
print(fake_news)

                                                   title  \
0      donald trump send embarrassing new year eve me...   
1      drunk bragging trump staffer start russian col...   
2      sheriff david clarke become internet joke thre...   
3        trump obsess even obama name code website image   
4        pope francis call donald trump christmas speech   
...                                                  ...   
23476  mcpain john mccain furious iran treat sailor well   
23477  justice yahoo settle e mail privacy class acti...   
23478  sunnistan us ally safe zone plan take territor...   
23479  blow 700 million al jazeera america finally ca...   
23480  10 u navy sailor hold iranian military sign ne...   

                                                    text      subject  \
0      donald trump wish americans happy new year lea...         News   
1      house intelligence committee chairman devin nu...         News   
2      friday reveal former milwaukee sheriff david c...    

In [33]:
from sklearn.model_selection import train_test_split

train_fake_X, t_fake_X, train_fake_y, t_fake_y = train_test_split(fake_news.drop('is_fake', axis=1), fake_news['is_fake'], test_size=0.2, random_state=101)
test_fake_X, val_fake_X, test_fake_y, val_fake_y = train_test_split(t_fake_X, t_fake_y, test_size=0.5, random_state=101)

train_true_X, t_true_X, train_true_y, t_true_y = train_test_split(true_news.drop('is_fake', axis=1), true_news['is_fake'], test_size=0.2, random_state=101)
test_true_X, val_true_X, test_true_y, val_true_y = train_test_split(t_true_X, t_true_y, test_size=0.5, random_state=101)

In [34]:
# Joining fake and true dataframes

train_X = pd.concat([train_fake_X, train_true_X])
val_X = pd.concat([val_fake_X, val_true_X])
test_X = pd.concat([test_fake_X, test_true_X])

train_y = pd.concat([train_fake_y, train_true_y])
val_y = pd.concat([val_fake_y, val_true_y])
test_y = pd.concat([test_fake_y, test_true_y])

In [46]:
train_X = train_X['text']
val_X = val_X['text']
test_X = test_X['text']

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features = 50000, lowercase=False, ngram_range=(1,2))
train_X_vec = vectorizer.fit_transform(train_X)
val_X_vec = vectorizer.transform(val_X)
test_X_vec = vectorizer.transform(test_X)

In [ ]:
val_X_vec.toarray().shape

Error: Session cannot generate requests